# 분석목표
- 중점관리 키워드,   저효율 키워드 추출
- 중점관리 광고그룹, 저효율 광고그룹 추출

# 분석과정:
1. 데이터전처리
1. 데이터탐색
1. 시각화
1. 데이터분석


## 1.데이터전처리
- read.excel()
- 데이터프레임 열단위 수치연산 및 데이터 타입 다루기

In [143]:
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform

try:
  if platform.system()=="Windows":
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
  else:
    rc('font', family='AppleGothic')
except:
  pass
matplotlib.rcParams['axes.unicode_minus'] = False


In [150]:
# 운영체제 디렉토리 경로 구분자가 다름
# - 윈도우 : 역슬래쉬(\)
# - 맥     : 슬래쉬(/)
df = pd.read_excel('./Data/naverAds.xls', skiprows=[0])
# df = pd.read_excel('C:\Repository\python_basic\4MachineLearning\Data\naverAds.xls, skiprows=[0]')
df.tail(1)
df.columns

Index(['광고그룹', '키워드', '노출수', '클릭수', '클릭률(%)', '평균클릭비용(VAT포함,원)',
       '총비용(VAT포함,원)'],
      dtype='object')

### 결측치 확인

In [156]:
df.isnull().sum(axis=1)

0       0
1       0
2       0
3       0
4       0
       ..
1376    0
1377    0
1378    0
1379    0
1380    0
Length: 1381, dtype: int64

### 데이터프레임의 열단위 수치연산 및 데이터타입 다루기
- 클릭수(반올림, 일의 자리로 변경)
- 클릭률
- 평균클릭비용(VAT포함, 원)
- 광고 KPI : 계산식
    - CTR(click through rate) : 클릭수/노출수 * 100
    - CPM(Cost Per Mile)      : 광고비용/노출수 * 1000
    - CPC(Cost Per Click)     : 광고비용/클릭수
    - CPA(Cost Per Action)    : 광고비용/구매수

In [157]:
# 클릭수 : 한번의 클릭이 총 몇번 이루어졌는지 나타내는 지표 => 반올림 처리
clk = round( df['클릭수'], 0)
clk.head(2)

0   9606.000000
1   8058.000000
Name: 클릭수, dtype: float64

In [161]:
df['클릭수'] = clk.astype(int)
df.head(2)

,광고그룹,키워드,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
0,올인원 패키지 : 디자인 툴_파워컨텐츠_포토샵,-,2319456,9606,0.414149,261.549448,2512444
1,올인원 패키지 : 업무자동화_VBA,-,767491,8058,1.049915,295.974808,2384965


In [40]:
# Click per mile
df['클릭률(%)'] = df['클릭수']/df['노출수']*100
df['클릭률(%)'].head(2)

0   0.414149
1   1.049915
Name: 클릭률(%), dtype: float64

In [163]:
# Cost per click 
cpc = round(df['평균클릭비용(VAT포함,원)'],0)
df['평균클릭비용(VAT포함,원)'] = cpc.astype(int)
df['평균클릭비용(VAT포함,원)'].head(1)

0    262
Name: 평균클릭비용(VAT포함,원), dtype: int32

In [164]:
df.describe()

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
count,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000
mean,35054.148443,52.504707,0.352871,981.624909,37919.580739
std,296465.029990,387.513479,1.000468,1372.401050,142149.155597
min,1001.000000,0.000000,0.000000,0.000000,0.000000
25%,1814.000000,1.000000,0.007152,64.000000,77.000000
50%,3805.000000,4.000000,0.061031,396.000000,2123.000000
75%,10734.000000,14.000000,0.237026,1303.000000,17435.000000
max,9626374.000000,9606.000000,13.587402,9363.000000,2512444.000000


In [166]:
pd.set_option('display.float_format', '{:2f}'.format) # 형식 float
df.describe()

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
count,1381.000000,1381.000000,1381.000000,1381.000000,1381.000000
mean,35054.148443,52.504707,0.352871,981.624909,37919.580739
std,296465.029990,387.513479,1.000468,1372.401050,142149.155597
min,1001.000000,0.000000,0.000000,0.000000,0.000000
25%,1814.000000,1.000000,0.007152,64.000000,77.000000
50%,3805.000000,4.000000,0.061031,396.000000,2123.000000
75%,10734.000000,14.000000,0.237026,1303.000000,17435.000000
max,9626374.000000,9606.000000,13.587402,9363.000000,2512444.000000


### 2.데이터탐색


### 중점관리 키워드 추출
- 노출수, 클릭수 기준 상위 5%에 해당하는 키워드 추출  
    - 95 백분위수 찾기 (quantile 함수 사용)  
    - 95 백분위수 이상의 노출수 추출  
    - 상위 5%에 해당하는 키워드 추출  

In [187]:
# 95 백분위 수 찾기
imp = df['노출수']
print( imp.quantile(), imp.median(), 
      imp.quantile(0), imp.min(), 
      imp.quantile(1), imp.max() )

3805.0 3805.0 1001.0 1001 9626374.0 9626374


In [174]:
imp = imp[ imp >= imp.quantile(0.95) ]

In [191]:
df_index = df.set_index('키워드')
df_index.head()

,광고그룹,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
키워드,,,,,,
-,올인원 패키지 : 디자인 툴_파워컨텐츠_포토샵,2319456,9606,0.414149,262,2512444
-,올인원 패키지 : 업무자동화_VBA,767491,8058,1.049915,296,2384965
일러스트,ㅍAOP 전체_중복키워드_디자인(삭제),1137840,324,0.028475,4842,1568699
-,올인원 패키지 : 데이터 분석 입문 온라인_파콘,694106,1864,0.268489,631,1175174
HTML,3. html,9626374,814,0.008452,1408,1145903


In [176]:
imp = df_index['노출수']
imp = imp[ imp >= imp.quantile(0.95) ]
imp.head()

키워드
-       2319456
-        767491
일러스트    1137840
-        694106
HTML    9626374
Name: 노출수, dtype: int64

In [177]:
clk = df_index['클릭수']
clk = clk[ clk >= clk.quantile(0.95) ]
clk.head()

키워드
-       9606
-       8058
일러스트     324
-       1864
HTML     814
Name: 클릭수, dtype: int32

In [194]:
# 노출수와 클릭수 모두 95 백분위수 이상 인 키워드
imp = df_index['노출수']
clk = df_index['클릭수']
result = df_index[ (imp >= imp.quantile(0.95)) & 
                  (clk >= clk.quantile(0.95)) ]
print( len(result.index), result.index[2] )
result.tail(2)

27 일러스트
                         광고그룹      노출수  클릭수   클릭률(%)  평균클릭비용(VAT포함,원)  \
키워드                                                                     
마블      ㅍAOP 전체_중복키워드_디자인(삭제)   907619  228 0.025121              265   
바이럴마케팅                 마케팅 전체  3095998  262 0.008450              220   

        총비용(VAT포함,원)  
키워드                   
마블             60533  
바이럴마케팅         57563  


### 저효율 광고그룹 추출
- 노출수, 클릭수 기준 상위 5%의 키워드가 아닌데도 비용이 많이 쓰이는 키워드
    - 노출수 95 백분위수 미만
    - 클릭수 95 백분위수 미만
    - 총비용 85 백분위수 이상
    - 총비용 95 백분위수 미만

In [183]:
cost = df_index['총비용(VAT포함,원)']
result = df_index[ (imp < imp.quantile(0.95)) & (clk < clk.quantile(0.95)) &
                 (cost >= cost.quantile(0.85)) & (cost < cost.quantile(0.95)) ]
print( len(result.index), result.index[3] )

97 부동산경매


### 광고그룹 분석
- as-is : 키워드 기준 분석
    - 키워드의 노출수/클릭수/총비용...
- to-bo : 광고그룹 기준분석
    - 데이터의 구성을 광고그룹 기준으로 변경
    - 광고그룹의 노출수/클릭수/총비용..
    - groupby함수 사용

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1381 entries, (올인원 패키지 : 디자인 툴_파워컨텐츠_포토샵, 2319456, 9606.0, 0.4141488348992178, 262, 2512444) to (4. 웹의 동작, 61193, 0.0, 0.0, 0, 0)
Data columns (total 7 columns):
광고그룹               1381 non-null object
키워드                1381 non-null object
노출수                1381 non-null int64
클릭수                1381 non-null float64
클릭률(%)             1381 non-null float64
평균클릭비용(VAT포함,원)    1381 non-null int32
총비용(VAT포함,원)       1381 non-null int64
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 80.9+ KB


In [204]:
grouped = df.groupby('광고그룹')
print( len(grouped) )
grouped.count()

186


,키워드,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,,
#000_마케팅 전체_main,48,48,48,48,48,48
001.BASIC DESIGN,9,9,9,9,9,9
001.데이터분석/ 통계,10,10,10,10,10,10
002.R,9,9,9,9,9,9
002.UX/UI,5,5,5,5,5,5
...,...,...,...,...,...,...
포토샵 유치원_일러스트레이터 유치원,13,13,13,13,13,13
프로그래밍 유치원,8,8,8,8,8,8
프로그래밍 전체,12,12,12,12,12,12


In [212]:
grouped['광고그룹'].unique()

광고그룹
#000_마케팅 전체_main                  [#000_마케팅 전체_main]
001.BASIC DESIGN                  [001.BASIC DESIGN]
001.데이터분석/ 통계                        [001.데이터분석/ 통계]
002.R                                        [002.R]
002.UX/UI                                [002.UX/UI]
                                     ...            
포토샵 유치원_일러스트레이터 유치원            [포토샵 유치원_일러스트레이터 유치원]
프로그래밍 유치원                                [프로그래밍 유치원]
프로그래밍 전체                                  [프로그래밍 전체]
프로그래밍 전체_파워컨텐츠_블록체인(삭제)    [프로그래밍 전체_파워컨텐츠_블록체인(삭제)]
프론트엔드 개발 스쿨 12기_기존 키워드      [프론트엔드 개발 스쿨 12기_기존 키워드]
Name: 광고그룹, Length: 186, dtype: object

In [104]:
grouped.mean()

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
#000_마케팅 전체_main,14780.145833,8.400000,0.183898,625.062500,7243.958333
001.BASIC DESIGN,5738.555556,6.800000,0.232924,298.666667,2427.333333
001.데이터분석/ 통계,3715.000000,20.280000,0.890571,1021.700000,19575.600000
002.R,13413.777778,8.533333,0.111199,360.000000,4796.000000
002.UX/UI,3090.800000,4.080000,0.089340,70.000000,550.000000
...,...,...,...,...,...
포토샵 유치원_일러스트레이터 유치원,72092.692308,16.707692,0.050832,648.384615,15781.615385
프로그래밍 유치원,5921.625000,7.800000,0.224738,2032.500000,23617.000000
프로그래밍 전체,90375.000000,9.200000,0.024037,123.916667,1626.166667


In [213]:
grouped.median()

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
#000_마케팅 전체_main,2765.000000,4.000000,0.055948,766.000000,2458.500000
001.BASIC DESIGN,5524.000000,2.000000,0.038716,358.000000,1023.000000
001.데이터분석/ 통계,1813.000000,13.500000,0.573846,1097.000000,13464.000000
002.R,5184.000000,4.000000,0.028016,119.000000,1287.000000
002.UX/UI,2649.000000,1.000000,0.090600,73.000000,88.000000
...,...,...,...,...,...
포토샵 유치원_일러스트레이터 유치원,5763.000000,4.000000,0.023327,853.000000,2706.000000
프로그래밍 유치원,3748.000000,3.000000,0.121833,1764.000000,8002.500000
프로그래밍 전체,3455.500000,1.000000,0.009331,130.500000,225.500000


In [214]:
grouped.sum()

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
#000_마케팅 전체_main,709447,401,8.827081,30003,347710
001.BASIC DESIGN,51647,60,2.096316,2688,21846
001.데이터분석/ 통계,37150,202,8.905710,10217,195756
002.R,120724,78,1.000794,3240,43164
002.UX/UI,15454,20,0.446702,350,2750
...,...,...,...,...,...
포토샵 유치원_일러스트레이터 유치원,937205,218,0.660812,8429,205161
프로그래밍 유치원,47373,62,1.797903,16260,188936
프로그래밍 전체,1084500,110,0.288445,1487,19514


In [215]:
df_group = grouped.sum()
df_group

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
#000_마케팅 전체_main,709447,401,8.827081,30003,347710
001.BASIC DESIGN,51647,60,2.096316,2688,21846
001.데이터분석/ 통계,37150,202,8.905710,10217,195756
002.R,120724,78,1.000794,3240,43164
002.UX/UI,15454,20,0.446702,350,2750
...,...,...,...,...,...
포토샵 유치원_일러스트레이터 유치원,937205,218,0.660812,8429,205161
프로그래밍 유치원,47373,62,1.797903,16260,188936
프로그래밍 전체,1084500,110,0.288445,1487,19514


### 결측치처리 fillna(0)

In [220]:
df_group['클릭률(%)']              = df_group['클릭수'] / df_group['노출수']
df_group['평균클릭비용(VAT포함,원)'] = df_group['총비용(VAT포함,원)']/df_group['클릭수']

In [221]:
df_group[ df_group['클릭수']==0 ].head(2)

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
Hyperledger Fabric 프로젝트 CAMP,1990,0,0.000000,nan,0
fin_camp_business_4,2136,0,0.000000,nan,0


In [222]:
df_group['평균클릭비용(VAT포함,원)'] = df_group['평균클릭비용(VAT포함,원)'].fillna(0)

In [225]:
df_group[ df_group['클릭수']==0 ].head(2)

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
Hyperledger Fabric 프로젝트 CAMP,1990,0,0.000000,0,0
fin_camp_business_4,2136,0,0.000000,0,0


### 반올림 후 정수화
- round()
- .astype(int)

In [224]:
df_group['평균클릭비용(VAT포함,원)'] = round( df_group['평균클릭비용(VAT포함,원)'], 0 )
df_group['평균클릭비용(VAT포함,원)'] = df_group['평균클릭비용(VAT포함,원)'].astype(int)
df_group[ df_group['클릭수']==0 ].head(2)

,노출수,클릭수,클릭률(%),"평균클릭비용(VAT포함,원)","총비용(VAT포함,원)"
광고그룹,,,,,
Hyperledger Fabric 프로젝트 CAMP,1990,0,0.000000,0,0
fin_camp_business_4,2136,0,0.000000,0,0


### 중점관리 광고그룹
- 중점관리 광고그룹
    - 상위 20% (노출수 80백분위수 이상)
    - 상위 10% (노출수 90백분위수 이상)

In [134]:
imp = df_group['노출수']
clk = df_group['클릭수']
result = df_group[( imp >= imp.quantile(0.8) ) &
                  ( clk >= clk.quantile(0.9) )]
result.index

Index(['#000_마케팅 전체_main', '1. 코딩', '3. html', '6. 파이썬', 'AOP 전체_중복키워드_디자인',
       'AOP 전체_중복키워드_비지니스', 'ㅍAOP 전체_중복키워드_디자인(삭제)', '마케팅 전체',
       '올인원 패키지 : 데이터 분석 입문 온라인_파콘', '올인원 패키지 : 디자인 툴_파워컨텐츠_포토샵',
       '올인원 패키지 : 업무자동화_VBA', '타이포그래피와 편집디자인', '프로그래밍 전체_파워컨텐츠_블록체인(삭제)'],
      dtype='object', name='광고그룹')

### 저효율 광고그룹
- 노출수, 클릭수 기준 상위의 키워드가 아닌데도 비용이 많이 쓰이는 그룹
    - 저효율 광고그룹
        - 하위 20% (노출수 80백분위수 미만)
        - 하위 20% (클릭수 90백분위수 미만)
        - 하위 20% (총비용 60백분위수 이상)
        - 하위 20% (총비용 90백분위수 미만)
        

In [139]:
cost = df_group['총비용(VAT포함,원)'] 
result = df_group[( imp < imp.quantile(0.8) ) & ( clk < clk.quantile(0.9) ) &
               ( cost >= cost.quantile(0.6) ) & ( cost < cost.quantile(0.9) )]

In [142]:
print( len(result.index), result.index[1] )

38 002.마케팅 입문


## 3.시각화


## 4.데이터분석
